In [1]:
import sys
import os
import fitsio
import subprocess
import numpy as np
import glob
import time
import random
import joblib
from esutil.pbar import PBar
from mattspy import BNLCondorParallel

In [2]:
import esutil

In [3]:
def _download_tile(tilename, cwd):
    os.system("mkdir -p data")

    d = fitsio.read(
        os.path.join(cwd, "fnames.fits"),
        lower=True,
    )
    tnames = np.array([
        d["filename"][i].split("_")[0]
        for i in range(d.shape[0])
    ])
    msk = tnames == tilename
    if np.sum(msk) != 4:
        return np.sum(msk)

    d = d[msk]
    mfiles = []
    for band in ["g", "r", "i", "z"]:
        msk = d["band"] == band
        if np.any(msk):
            _d = d[msk]
            for i in range(len(_d)):
                fname = os.path.join(d["path"][msk][i], d["filename"][msk][i])
                if not os.path.exists("./data/%s" % os.path.basename(fname)):
                    cmd = """\
rsync \
-av \
--password-file $DES_RSYNC_PASSFILE \
${DESREMOTE_RSYNC_USER}@${DESREMOTE_RSYNC}/%s \
./data/%s
""" % (fname, os.path.basename(fname))
                    subprocess.run(cmd, shell=True, check=True)
            mfiles.append("./data/%s" % os.path.basename(fname))

    return mfiles

In [4]:
def _extract_expnum_ccdnum(ii):
    expnums = np.zeros(len(ii), dtype=int) + -1
    ccdnums = np.zeros(len(ii), dtype=int) + -1
    for i in range(len(ii)):
        ip = ii["image_path"][i]
        if len(ip) > 0:
            expnums[i] = int(os.path.basename(ip).split("_")[0][1:])
            ccdnums[i] = int(os.path.basename(ip).split("_")[2][1:])
    return expnums, ccdnums

def _process_file(fname, tname, band):
    ei = fitsio.read(fname, ext="epochs_info")
    ii = fitsio.read(fname, ext="image_info")
    ei = esutil.numpy_util.add_fields(
        ei, 
        [
            ("tilename", "U12"), 
            ("band", "U1"), 
            ("expnum", "i8"), 
            ("ccdnum", "i4"),
            ("image_flags", "i4"),
        ],
    )
    ei["tilename"] = tname
    ei["band"] = band
    
    expnums, ccdnums = _extract_expnum_ccdnum(ii)
    ei["expnum"] = expnums[ei["image_id"]]
    ei["ccdnum"] = ccdnums[ei["image_id"]]
    ei["image_flags"] = ii["image_flags"][ei["image_id"]]
    assert np.all(ei["ccdnum"] <= 62)
    assert np.array_equal(ii["image_id"], np.arange(len(ii)))
    
    try:
        os.remove(fname)
    except Exception:
        pass
        
    return ei, ii

In [5]:
def _process_tile(tname, uccds):
    fnames = _download_tile(tname, os.getcwd())
    for fname in fnames:
        band = os.path.basename(fname).split("_")[2]
        tname = os.path.basename(fname).split("_")[0]
        assert band in ["g", "r", "i", "z"]
        ei, _ = _process_file(fname, tname, band)

        ofname = os.path.join(
            "epochs_info", 
            band, 
            os.path.basename(fname)[:-len("pizza-cutter-slices.fits.fz")] + "epochs-info.fits",
        )

        fitsio.write(ofname, ei, clobber=True)
        for i in range(len(ei)):
            uccds.add((ei["expnum"][i], ei["ccdnum"][i], ei["band"][i]))

In [6]:
tnames = np.unique(np.array([
    fn.split("_")[0]
    for fn in fitsio.read("fnames.fits")["FILENAME"]
]))

for band in "griz":
    os.makedirs("./epochs_info/"+ band, exist_ok=True)

uccds = set()
for tname in esutil.pbar.PBar(tnames):
    _process_tile(tname, uccds)
    
    if tname == tnames[3]:
        break

|--------------------| 2/10169   0% [elapsed: 00:04 left: 6:09:16]


    DES rsync daemon

receiving incremental file list
DES0000+0335_r5932p01_r_pizza-cutter-slices.fits.fz

sent 42 bytes  received 934707647 bytes  89019779.90 bytes/sec
total size is 934479360  speedup is 1.00

    DES rsync daemon

receiving incremental file list
DES0000+0335_r5932p01_i_pizza-cutter-slices.fits.fz

sent 42 bytes  received 901905079 bytes  94937381.16 bytes/sec
total size is 901684800  speedup is 1.00

    DES rsync daemon

receiving incremental file list
DES0000+0335_r5932p01_z_pizza-cutter-slices.fits.fz

sent 42 bytes  received 933950015 bytes  98310532.32 bytes/sec
total size is 933721920  speedup is 1.00


|--------------------| 3/10169   0% [elapsed: 00:34 left: 32:26:45]


    DES rsync daemon

receiving incremental file list
DES0000+0418_r5932p01_g_pizza-cutter-slices.fits.fz

sent 42 bytes  received 905485791 bytes  95314298.21 bytes/sec
total size is 905264640  speedup is 1.00

    DES rsync daemon

receiving incremental file list
DES0000+0418_r5932p01_r_pizza-cutter-slices.fits.fz

sent 42 bytes  received 903002623 bytes  78521970.87 bytes/sec
total size is 902782080  speedup is 1.00

    DES rsync daemon

receiving incremental file list
DES0000+0418_r5932p01_i_pizza-cutter-slices.fits.fz

sent 42 bytes  received 858585063 bytes  90377379.47 bytes/sec
total size is 858375360  speedup is 1.00

    DES rsync daemon

receiving incremental file list
DES0000+0418_r5932p01_z_pizza-cutter-slices.fits.fz

sent 42 bytes  received 894841591 bytes  94193856.11 bytes/sec
total size is 894623040  speedup is 1.00


In [9]:
import glob

for band in "griz":
    fns = glob.glob("epochs_info/%s/*.fits" % band)
    uccds = set()
    for fn in fns:
        ei = fitsio.read(fn)
        for i in range(len(ei)):
            uccds.add((ei["expnum"][i], ei["ccdnum"][i], ei["band"][i]))        
            
    with open("uccds_%s.txt" % band, "w") as fp:
        fp.write("# expnum ccdnum band\n")
        for dta in uccds:
            fp.write("%d %d %s\n" % dta)

In [ ]:
ei = fitsio.read("./data/DES0124-3415_r5933p01_g_pizza-cutter-slices.fits.fz", ext="epochs_info")
ii = fitsio.read("./data/DES0124-3415_r5933p01_g_pizza-cutter-slices.fits.fz", ext="image_info")

In [ ]:
_extract_expnum_ccdnum(ii)[1]

In [ ]:
ii[np.unique(ei["image_id"])]["image_path"][0].split("/")[1].split("_")[0][1:])

In [ ]:
[os.path.basename(ip).split("_")[0][1:] for ip in ii["image_path"]]

In [ ]:
ei.dtype.names, ii.dtype.names

In [ ]:
ei = esutil.numpy_util.add_fields(ei, [("tilename", "U12"), ("band", "U1"), ("expnum", "i4"), ("ccdnum", "i4")])

In [ ]:
ei["tilename"] = "DES0124-3415"
ei["band"] = "g"

In [ ]:
int(ii[np.unique(ei["image_id"])]["image_path"][0].split("/")[1].split("_")[0][1:])

In [ ]:
ii.dtype.names

In [ ]:
def _run_tile(tilename, seed, opth, tmpdir, cwd, cpus):
    os.system("mkdir -p ./data")
    for _ in range(10):
        try:
            mfiles = _download_tile(tilename, cwd)
            break
        except Exception:
            time.sleep(600 + random.randint(-100, 100))
            mfiles = None
            pass

    if mfiles is None:
        raise RuntimeError("Could not download files for tile %s" % tilename)
    elif not isinstance(mfiles, list):
        raise RuntimeError("Only found %d files for tile %s" % (
            mfiles, tilename
        ))

    if tmpdir is None:
        tmpdir = os.environ["TMPDIR"]

    cmd = """\
run-metadetect-on-slices \
  --config=%s/metadetect-v8.yaml \
  --output-path=./mdet_data \
  --seed=%d \
  --use-tmpdir \
  --tmpdir=%s \
  --log-level=INFO \
  --n-jobs=%d \
  --band-names=griz %s %s %s %s""" % (
        cwd, seed, tmpdir, cpus, mfiles[0], mfiles[1], mfiles[2], mfiles[3],
    )
    subprocess.run(cmd, shell=True, check=True)

    fnames = glob.glob("./mdet_data/%s*" % tilename)
    for fname in fnames:
        pth1 = os.path.realpath(os.path.abspath(fname))
        pth2 = os.path.realpath(os.path.abspath(
            os.path.join(opth, os.path.basename(fname)))
        )
        if pth1 != pth2:
            subprocess.run(
                "mv %s %s" % (pth1, pth2),
                shell=True,
                check=True,
            )

    for mfile in mfiles:
        try:
            os.remove(mfile)
        except Exception:
            pass


cwd = os.path.abspath(os.path.realpath(os.getcwd()))
seed = 100
os.system("mkdir -p ./mdet_data")
os.system("chmod go-rw ./mdet_data")
opth = os.path.abspath("./mdet_data")

if len(sys.argv) == 1:
    d = fitsio.read(
        os.path.join(cwd, "fnames.fits"),
        lower=True,
    )
    tnames = sorted(list(set([
        d["filename"][i].split("_")[0]
        for i in range(d.shape[0])
    ])))
    rng = np.random.RandomState(seed=seed)
    seeds = rng.randint(low=1, high=2**29, size=len(tnames))
    tmpdir = None
    cpus = 4
else:
    tnames = [sys.argv[1]]
    tmpdir = "/data/beckermr/tmp/" + tnames[0] + "_mdet"
    os.system("mkdir -p " + tmpdir)
    cpus = 1

if len(tnames) == 1:
    _run_tile(tnames[0], seed, opth, tmpdir, cwd, cpus)
else:
    d = fitsio.read(
        os.path.join(cwd, "fnames.fits"),
        lower=True,
    )
    all_tnames = np.array([
        d["filename"][i].split("_")[0]
        for i in range(d.shape[0])
    ])
    for mem in [8]:
        with BNLCondorParallel(verbose=0, mem=mem, cpus=cpus) as exc:
            jobs = []
            for tilename, seed in PBar(
                zip(tnames, seeds), total=len(tnames), desc="making jobs"
            ):
                if (
                    np.sum(all_tnames == tilename) == 4
                    and len(
                        glob.glob("%s/%s*.fits.fz" % (opth, tilename))
                    ) == 0
                ):
                    jobs.append(
                        joblib.delayed(_run_tile)(
                            tilename, seed, opth, tmpdir, cwd, cpus,
                        )
                    )

            for res in PBar(exc(jobs), total=len(jobs), desc="running mdet"):
                try:
                    res.result()
                except Exception as e:
                    print("ERROR: " + repr(e), flush=True)